In [29]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import eli5

import regex as re
import nltk
#nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier


from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier

from nltk.corpus import stopwords

import pickle
import sys

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# df_reddit = pickle.load(open('../DataSet/df_reddit_for_model.pkl', 'rb'))
df_reddit = pickle.load(open('../DataSet/df_reddit_for_model.pkl', 'rb'))
EDA_df = pickle.load(open('../DataSet/EDA_all_for_model.pkl', 'rb'))
# df_reddit['target'] = df_reddit['subreddit'].replace({"NASA": 1, "Space_discussion": 0})
df_reddit.head(2)

,ent,text_merged,word_count*,sentiment_score*,target,polarity_score,polarity,polarity_VSA,lower_sent,tagged_sent,text_complexity
0,1,star shine saturn ring,4,0.0,1,0.0,1,1,star shine saturn ring,"[(star, NN), (shine, NN), (saturn, NN), (ring,...",92.80
1,2,smithsonian nation air space museum,5,0.0,1,0.0,1,1,smithsonian nation air space museum,"[(smithsonian, JJ), (nation, NN), (air, NN), (...",15.64


In [3]:
EDA_df.head(2)

,target,'',CC,CD,DT,EX,FW,IN,JJ,JJR,...,WP,WP$,WRB,ent,word_count*,sentiment_score*,polarity_score,polarity,polarity_VSA,text_complexity
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,4,0.0,0.0,1,1,92.80
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2,5,0.0,0.0,1,1,15.64


Defining X and Y variables and use train test split function

In [4]:
X = df_reddit['text_merged']
y = df_reddit['target']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.25,
                                                    stratify=y,
                                                    random_state = 101)

In [5]:
cvec = CountVectorizer(min_df=4, max_df=1.0,
                       ngram_range=(1,2),max_features = 3000)

In [6]:
term_mat = cvec.fit_transform(df_reddit['text_merged'])

In [7]:
term_df = pd.DataFrame(term_mat.toarray(), 
                       columns=cvec.get_feature_names())

term_df.head(2)

,aa,ab,abil,abl,abl get,abl see,aboard,aboard intern,aboard space,abort,...,york,youlikebet,young,youtu,zealand,zero,zero graviti,zone,zoom,zubrin
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Merge two base DFs and apply ML algorithms

In [8]:
EDA_df.head(2)

,target,'',CC,CD,DT,EX,FW,IN,JJ,JJR,...,WP,WP$,WRB,ent,word_count*,sentiment_score*,polarity_score,polarity,polarity_VSA,text_complexity
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,4,0.0,0.0,1,1,92.80
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2,5,0.0,0.0,1,1,15.64


In [9]:
for col in EDA_df.columns:
    if col == 'target':
        print(col)


target


In [10]:
EDA_df.columns

Index(['target', '''', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS',
       'MD', 'NN', 'NNP', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR',
       'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ',
       'WDT', 'WP', 'WP$', 'WRB', 'ent', 'word_count*', 'sentiment_score*',
       'polarity_score', 'polarity', 'polarity_VSA', 'text_complexity'],
      dtype='object')

In [11]:
term_df.head(2)

,aa,ab,abil,abl,abl get,abl see,aboard,aboard intern,aboard space,abort,...,york,youlikebet,young,youtu,zealand,zero,zero graviti,zone,zoom,zubrin
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
for col in term_df.columns:
    if col == 'target':
        print(col)

target


In [13]:
term_df = term_df.drop(columns = ['target'])

In [14]:
All_DF =  pd.concat([EDA_df, term_df], axis=1)
All_DF.head(2)

,target,'',CC,CD,DT,EX,FW,IN,JJ,JJR,...,york,youlikebet,young,youtu,zealand,zero,zero graviti,zone,zoom,zubrin
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
All_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Columns: 3042 entries, target to zubrin
dtypes: float64(38), int64(3004)
memory usage: 278.5 MB


In [16]:
y2 = All_DF['target']
X2 = All_DF.drop(columns = ['target'])

In [17]:
X_arr2 = np.array(X2)
y_arr2 = np.array(y2)

In [18]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_arr2,
                                                        y_arr2,
                                                        test_size = 0.25,
                                                        random_state = 101)

#### LR model

In [25]:
lr = LogisticRegression(solver='sag', max_iter=1000, 
                        verbose=2, n_jobs=-1)

lr.fit(X_train2, y_train2);

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.


max_iter reached after 154 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.6min finished


In [26]:
lr.score(X_train2, y_train2); 
print('train score', lr.score(X_train2, y_train2))

lr.score(X_test2, y_test2); 
print('test score', lr.score(X_test2, y_test2))

train score 0.834
test score 0.8286666666666667


In [27]:
preds = lr.predict(X_test2)

# Save confusion matrix values
tn, fp, fn, tp = confusion_matrix(y_test2, preds).ravel()

# View confusion matrix

#plot_confusion_matrix(gs, X_test_features, y_test, cmap='Blues', values_format='d');
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Precision = tp / (tp + fp)
Recall = tp / (tp + fn)

Accuracy, Precision, Recall

(0.8286666666666667, 0.9643527204502814, 0.6835106382978723)

Save the lr model.

In [31]:
# save the model to disk
sys.path.insert(1, '../Codes/')
filename = '../Codes/Models/lr_All_DF_P04_02.sav'
pickle.dump(lr, open(filename, 'wb'))

#### ANN Grid Search with all CPUs

In [25]:
mlp = MLPClassifier(hidden_layer_sizes=50, activation='relu', solver='adam', 
                    alpha=0.0001, batch_size='auto', learning_rate='constant', 
                    learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, 
                    random_state=None, tol=0.0001, verbose=False, warm_start=False, 
                    momentum=0.9, nesterovs_momentum=True, early_stopping=False, 
                    validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, 
                    n_iter_no_change=10, max_fun=15000)


mlp_params = {
    'hidden_layer_sizes': [100,150],
    'activation': ['tanh', 'relu']}
mlp = GridSearchCV(mlp, param_grid=mlp_params, cv=3, 
                   verbose=1, n_jobs=-1)
mlp.fit(X_train2, y_train2)

print(mlp.best_score_)
mlp.best_params_

Fitting 3 folds for each of 4 candidates, totalling 12 fits
0.8441111111111113


{'activation': 'relu', 'hidden_layer_sizes': 150}

#### ANN model

In [24]:
mlp = MLPClassifier(hidden_layer_sizes=100, activation='relu', solver='adam', 
                    alpha=0.0001, batch_size='auto', learning_rate='constant', 
                    learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, 
                    random_state=None, tol=0.0001, verbose=False, warm_start=False, 
                    momentum=0.9, nesterovs_momentum=True, early_stopping=False, 
                    validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, 
                    n_iter_no_change=10, max_fun=15000)


mlp.fit(X_train2, y_train2)
print(mlp.score(X_test2, y_test2))

preds = mlp.predict(X_test2)

# Save confusion matrix values
tn, fp, fn, tp = confusion_matrix(y_test2, preds).ravel()

# View confusion matrix

#plot_confusion_matrix(gs, X_test_features, y_test, cmap='Blues', values_format='d');
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Precision = tp / (tp + fp)
Recall = tp / (tp + fn)

Accuracy, Precision, Recall

0.849


(0.849, 0.8375080282594733, 0.8670212765957447)

Save the ANN model.

In [34]:
# save the model to disk
filename = 'mlp_relu_adam_150_layers_finalized_model.sav'
pickle.dump(mlp, open(filename, 'wb'))

#### GBoost Grid Search with all CPUs

In [26]:
gboost = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, 
                                    n_estimators=100, subsample=1.0, criterion='friedman_mse', 
                                    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                    max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, 
                                    init=None, random_state=None, max_features=None, verbose=0, 
                                    max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, 
                                    n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
gboost_params = {
    'max_depth': [2,3,4],
    'n_estimators': [100, 250, 500],
    'learning_rate': [.08, .1, .12]
}
gs = GridSearchCV(gboost, param_grid=gboost_params, 
                  cv=3, verbose=1, n_jobs=-1)
gs.fit(X_train2, y_train2)

print(gs.best_score_)
gs.best_params_

Fitting 3 folds for each of 27 candidates, totalling 81 fits
0.8782222222222223


{'learning_rate': 0.08, 'max_depth': 4, 'n_estimators': 500}

### The grid search for gboost algorithm was ran and the results were provided below.
```python
gboost = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, 
                                    n_estimators=100, subsample=1.0, criterion='friedman_mse', 
                                    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                    max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, 
                                    init=None, random_state=None, max_features=None, verbose=0, 
                                    max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, 
                                    n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
gboost_params = {
    'max_depth': [2,3,4],
    'n_estimators': [100, 250, 500],
    'learning_rate': [.08, .1, .12]
}
gs = GridSearchCV(gboost, param_grid=gboost_params, 
                  cv=3, verbose=1, n_jobs=-1)
gs.fit(X_train2, y_train2)

print(gs.best_score_)
gs.best_params_
```

### Results

```
Fitting 3 folds for each of 27 candidates, totalling 81 fits
0.8782222222222223
{'learning_rate': 0.08, 'max_depth': 4, 'n_estimators': 500}
```



going with the best gboost parameters as follow.

In [27]:
gboost = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, 
                                    n_estimators=500, subsample=1.0, criterion='friedman_mse', 
                                    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                    max_depth=4, min_impurity_decrease=0.0, min_impurity_split=None, 
                                    init=None, random_state=None, max_features=None, verbose=True, 
                                    max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, 
                                    n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

gboost.fit(X_train2, y_train2)

print(gboost.score(X_test2, y_test2))

preds = gboost.predict(X_test2)

# Save confusion matrix values
tn, fp, fn, tp = confusion_matrix(y_test2, preds).ravel()

# View confusion matrix

#plot_confusion_matrix(gs, X_test_features, y_test, cmap='Blues', values_format='d');
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Precision = tp / (tp + fp)
Recall = tp / (tp + fn)

Accuracy, Precision, Recall

0.877


(0.877, 0.927012791572611, 0.8191489361702128)

Save the gboost model.

In [35]:
# save the model to disk
filename = 'gboost_learning_rate_0.08_depth_4_estimators_500_finalized_model.sav'
pickle.dump(gboost, open(filename, 'wb'))

This is the best results so far we got. let's increase the number of estimators and run it again.

In [28]:
gboost = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, 
                                    n_estimators=1200, subsample=1.0, criterion='friedman_mse', 
                                    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                    max_depth=4, min_impurity_decrease=0.0, min_impurity_split=None, 
                                    init=None, random_state=None, max_features=None, verbose=True, 
                                    max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, 
                                    n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

gboost.fit(X_train2, y_train2)

print(gboost.score(X_test2, y_test2))

preds = gboost.predict(X_test2)

# Save confusion matrix values
tn, fp, fn, tp = confusion_matrix(y_test2, preds).ravel()

# View confusion matrix

#plot_confusion_matrix(gs, X_test_features, y_test, cmap='Blues', values_format='d');
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Precision = tp / (tp + fp)
Recall = tp / (tp + fn)

Accuracy, Precision, Recall

      Iter       Train Loss   Remaining Time 
         1           1.2731           30.29m
         2           1.1806           31.30m
         3           1.1040           31.54m
         4           1.0394           31.58m
         5           0.9851           31.67m
         6           0.9386           31.48m
         7           0.8987           31.36m
         8           0.8644           31.52m
         9           0.8349           32.07m
        10           0.8091           31.99m
        20           0.6728           32.54m
        30           0.6232           32.40m
        40           0.6015           34.29m
        50           0.5866           35.76m
        60           0.5751           36.41m
        70           0.5640           36.89m
        80           0.5551           37.21m
        90           0.5466           37.42m
       100           0.5391           37.38m
       200           0.4800           35.20m
       300           0.4347           32.10m
       40

(0.879, 0.9204126750184229, 0.8304521276595744)

Almost No improvment in model estimation. Lets save the model.

In [32]:
# save the model to disk
filename = 'gboost_1200_finalized_model.sav'
pickle.dump(gboost, open(filename, 'wb'))
 
# # some time later...
 
# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

In [37]:
gboost = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, 
                                    n_estimators=2500, subsample=1.0, criterion='friedman_mse', 
                                    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                    max_depth=6, min_impurity_decrease=0.0, min_impurity_split=None, 
                                    init=None, random_state=None, max_features=None, verbose=True, 
                                    max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, 
                                    n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

gboost.fit(X_train2, y_train2)

print(gboost.score(X_test2, y_test2))

preds = gboost.predict(X_test2)

# Save confusion matrix values
tn, fp, fn, tp = confusion_matrix(y_test2, preds).ravel()

# View confusion matrix

#plot_confusion_matrix(gs, X_test_features, y_test, cmap='Blues', values_format='d');
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Precision = tp / (tp + fp)
Recall = tp / (tp + fn)

Accuracy, Precision, Recall

      Iter       Train Loss   Remaining Time 
         1           1.2693           89.16m
         2           1.1735           89.21m
         3           1.0936           89.60m
         4           1.0265           89.56m
         5           0.9698           89.21m
         6           0.9204           88.53m
         7           0.8786           88.65m
         8           0.8413           89.66m
         9           0.8098           91.27m
        10           0.7822           89.73m
        20           0.6319           91.94m
        30           0.5762           94.46m
        40           0.5507           96.50m
        50           0.5334           99.96m
        60           0.5192          103.26m
        70           0.5052          104.18m
        80           0.4942          107.42m
        90           0.4854          107.20m
       100           0.4774          109.55m
       200           0.4010          116.83m
       300           0.3513          115.00m
       40

(0.8773333333333333, 0.9109985528219972, 0.8371010638297872)

In [39]:
# save the model to disk
filename = 'gboost_2500_depth_6_finalized_model.sav'
pickle.dump(gboost, open(filename, 'wb'))
 
# # some time later...
 
# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)